In [55]:
# Importing useful libraries
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, \
                    LeaveOneOut, KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
import sklearn.utils

from imblearn.over_sampling import SMOTE

import xgboost
import lightgbm

import warnings
warnings.filterwarnings('ignore')

In [56]:
# Loading the dataset
df = pd.read_csv('./Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [57]:
# Checking the shape of the dataset
df.shape

(10000, 14)

In [58]:
# Checking for missing values
df.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [59]:
# Dropping 'stab' column as instructed
df = df.drop('stab', axis=1)
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [60]:
y = df.pop('stabf') # Pops out the stabf column as the label
X = df # Uses the remaining columns as the features
X.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923


In [61]:
# Splitting the data into 80:20 training and testing test with a random_state of 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) 

In [62]:
scaler = StandardScaler() # Initializes a StandardScaler object
scaled_X_train = scaler.fit_transform(X_train) # Fits and transform the training set
scaled_X_test = scaler.transform(X_test) # Transforms the testing set

#### Let's create a function that returns the metric score of a test set. The metric can be any of  accuracy_score, precision_score, recall_score, f1_score and confusion matrix.

In [63]:
# First, let's create a dictionary of the metrices we will be using
metrics = {'accuracy_score': accuracy_score, 'precision_score': precision_score, 'recall_score': recall_score, 
               'f1_score': f1_score, 'confusion_matrix': confusion_matrix}

In [64]:
# Defining the function
def get_metric_score(metric, ytrue, ypred, neg_pos_label):
    ''' This function returns the specified metric score. It only works with classifier metrics.
        
        Args:   metric (string): the evaluating metric, can be any of accuracy_score, precision_score, recall_score, f1_score, 
                                 or confusion matrix.
                ytrue (array): the true labels
                ypred (array): the predicted labels
                neg_pos_label (list): a list of the classes you want as the negative and positive label in order 
                                      of [negative_label, positive_label]
                
        Return: returns the metric score
    '''
    
    if metric == 'accuracy_score':
        return accuracy_score(ytrue, ypred)
    
    elif metric == 'confusion_matrix':
        return confusion_matrix(ytrue, ypred, neg_pos_label)
    
    else:
        return metrics[metric](ytrue, ypred, pos_label=neg_pos_label[1]) # this is done because precision, recall and f1_score
                                                                         # takes the same arguments

#### Let's create a that function fits a classifier on a training set and prints out the accuracy_score, precision_score, recall_score,    f1_score and confusion matrix of the testing set.

In [65]:
# Defining the function
def fit_and_score(classifier, xtrain, ytrain, xtest, ytest, neg_pos_label):
    ''' This function fits a classifier on a training set and prints out the accuracy_score, precision_score, recall_score, 
    f1_score and confusion matrix of the testing set.
    
    Args: classifier (classifier object): the classifier you want to use
          xtrain (ndarray): the training features
          ytrain (array): the training labels
          xtest (ndarray): the testing features
          ytest (array): the testing labels
          neg_pos_label (list): a list of the classes you want as the negative and positive label in order 
                                      of [negative_label, positive_label]
    '''
    classifier.fit(xtrain, ytrain) # fits the classifier
    ypred = classifier.predict(xtest) # predicts
    
    # for each metric in metrics (dictionary earlier defined), print out the metric score.
    for metric in metrics:
        
        # this 'if' block is to ensure that the confusion matrix is properly printed out to improve redability
        if metric == 'confusion_matrix':
            print()
            print('confusion_matrix is:')
            print(get_metric_score(metric, ytest, ypred, neg_pos_label))
            
        else:
            print('{} is {}'.format(metric, get_metric_score(metric, ytest, ypred, neg_pos_label)))

In [66]:
label_list = ['unstable', 'stable'] # this is to be used as the neg_pos_label needed in fit_and_score function

## Let's evaluate our model on different classifiers

#### Training and testing on RandomForestClassifier

In [67]:
random_forest = RandomForestClassifier(random_state=1)
fit_and_score(random_forest, scaled_X_train, y_train, scaled_X_test, y_test, label_list)

accuracy_score is 0.929
precision_score is 0.9191176470588235
recall_score is 0.8778089887640449
f1_score is 0.8979885057471264

confusion_matrix is:
[[1233   55]
 [  87  625]]


In [ ]:
extra_trees = ExtraTreesClassifier(random_state=1)
fit_and_score(extra_trees, scaled_X_train, y_train, scaled_X_test, y_test, label_list)

In [ ]:
xgb = xgboost.XGBClassifier(random_state=1)
fit_and_score(xgb, scaled_X_train, y_train, scaled_X_test, y_test, label_list)

In [ ]:
lgbm = lightgbm.LGBMClassifier(random_state=1)
fit_and_score(lgbm, scaled_X_train, y_train, scaled_X_test, y_test, label_list)

In [ ]:
# initializing search space of hyperparameters
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

# making a dictionary of the grid
hyperparameter_grid = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split, 'max_features': max_features}

In [ ]:
extra_trees2 = ExtraTreesClassifier(random_state=1) # initializes an ExtraTreesClassifier

# initializing a RandomizedSearchCV
tuned_extra_trees = RandomizedSearchCV(extra_trees2, hyperparameter_grid, random_state=1, verbose=1, n_jobs=3) 

In [ ]:
fit_and_score(tuned_extra_trees, scaled_X_train, y_train, scaled_X_test, y_test, label_list)

In [ ]:
random_forest2 = RandomForestClassifier(random_state=1) # initializes a random forest classifier object
random_forest2.fit(scaled_X_train, y_train) # fits the model
ypred_random_forest = random_forest2.predict(scaled_X_test) # predicts on the testing set
accuracy_score(y_test, ypred_random_forest) # ouputs the accuracy

In [ ]:
xgb2 = xgboost.XGBClassifier(random_state=1) # initializes a xgboost classifier object
xgb2.fit(scaled_X_train, y_train) # fits the model
ypred_xgb = xgb2.predict(scaled_X_test) # predicts on the testing set
accuracy_score(y_test, ypred_xgb) # ouputs the accuracy

In [ ]:
light_gbm2 = lightgbm.LGBMClassifier(random_state=1) # initializes a light gbm classifier object
light_gbm2.fit(scaled_X_train, y_train) # fits the model
ypred_light_gbm2 = light_gbm2.predict(scaled_X_test) # predicts on the testing set
accuracy_score(y_test, ypred_light_gbm2) # ouputs the accuracy

In [ ]:
# initializing search space of hyperparameters
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None] 

# making a dictionary of the grid
hyperparameter_grid = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf, 
                       'min_samples_split': min_samples_split, 'max_features': max_features}

In [ ]:
extra_trees2 = ExtraTreesClassifier(random_state=1) # initializes an ExtraTreesClassifier object

# initializing a RandomizedSearchCV object
tuned_extra_trees2 = RandomizedSearchCV(extra_trees2, hyperparameter_grid, cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, 
                                        verbose = 1, random_state = 1)

In [ ]:
# fitting the tuned_extra_trees2 model
tuned_extra_trees2.fit(scaled_X_train, y_train)

In [ ]:
# finding the best hyperparameters
tuned_extra_trees2.best_params_

In [ ]:
# let's train an ordinary extra trees classifier
extra_trees3 = ExtraTreesClassifier(random_state=1) # initializes an extra trees classifier object
extra_trees3.fit(scaled_X_train, y_train) # fits the extra trees

In [ ]:
ypred_ordinary = extra_trees3.predict(scaled_X_test) # predicts on the testing set

print('accuracy of ordinary extra trees is {}'.format(accuracy_score(y_test, ypred_ordinary)))

In [ ]:
# now let's find the accuracy of the tuned extra trees
ypred_tuned = tuned_extra_trees2.best_estimator_.predict(scaled_X_test) # predicts with the best estimator of the tuned extra 
                                                                        # trees
print('accuracy of tuned extra trees is {}'.format(accuracy_score(y_test, ypred_tuned)))

In [ ]:
features_importance = tuned_extra_trees2.best_estimator_.feature_importances_ # finds the feature importance

In [ ]:
most_important_feature = features_importance.max() # finds the most important feature
least_important_feature = features_importance.min() # finds the least important feature

cols = X.columns # assigns the features in the data to a variable 'cols'

In [ ]:
print('most important feature is {}'.format(cols[features_importance == most_important_feature][0]))

In [ ]:
print('least important feature is {}'.format(cols[features_importance == least_important_feature][0]))